# Circonferenza Julia

Vettore di Posizione:
$$
    P(t) = \sum_{i=1}^{n+1}B_iN_{i,k}(t), \qquad 2 \leq k \leq n+1
$$
dove `N` indica la funzione di base:
$$
    N_{i,k}(t) = \frac{(t-x_i)N_{i,k-1}(t)}{x_{i+k-1}-x_i} + \frac{(x_{i+k}-t)N_{i+1,k-1}(t)}{x_{i+k}-x_{i+1}}
$$


In [4]:
B = [0 0;0 2;2 2];
knotVec = [0 0 0 1 1 1];

In [5]:
function P(t::Int64)
    partial 
    for i = 1 : n+1
        
end

P (generic function with 1 method)

In [8]:
using Plots

INFO: Precompiling module Plots.


In [15]:
function basis(c::Int64, t::Float64, npts::Int64, x::Array{Int64})::Array{Float64}
    temp = Array{Float64}(36);
    n = Float64[];
    nplusc = npts + c;
    
#    print(knot vector is );
#    for i=1:nplusc
#        print(& &, i, x[i]);
#    end
#    print(t is &, t);
    
    # calculate the first order basis functions n[i][1]
    
    for i=1:nplusc-1
        if (t>=x[i])&&(t<x[i+1])
            temp[i] = 1;
        else
            temp[i] = 0;
        end
    end
    
    # calculate the higher order basis functions
    for k=2:c
        for i=1:nplusc-k
            if temp[i] != 0    # if the lower order basis function is zero skip the calculation
                d = ((t-x[i])*temp[i])/(x[i+k-1]-x[i]);
            else
                d = 0;
            end
            if temp[i+1] != 0    # if the lower order basis function is zero skip the calculation
                e = ((x[i+k]-t)*temp[i+1])/(x[i+k]-x[i+1]);
            else
                e = 0;
            end
            temp[i] = d+e;
        end
    end
    
    if t == x[nplusc]    # pick the last point
        temp[npts] = 1;
    end
    
    # put in n array
    for i=1:npts
        push!(n, temp[i]);
    end
    return n;
end

basis (generic function with 1 method)

In [17]:
basis(2, 0.2, 3, knotVec)

3-element Array{Float64,1}:
 0.0
 0.8
 0.2

In [18]:
?plot

search: plot plot! plotly plot3d Plots plot3d! plotlyjs plotattr plotarea



The main plot command. Use `plot` to create a new plot object, and `plot!` to add to an existing one:

```
    plot(args...; kw...)                  # creates a new plot window, and sets it to be the current
    plot!(args...; kw...)                 # adds to the `current`
    plot!(plotobj, args...; kw...)        # adds to the plot `plotobj`
```

There are lots of ways to pass in data, and lots of keyword arguments... just try it and it will likely work as expected. When you pass in matrices, it splits by columns. To see the list of available attributes, use the `plotattr([attr])` function, where `attr` is the symbol `:Series:`, `:Subplot:`, `:Plot` or `:Axis`. Pass any attribute to `plotattr` as a String to look up its docstring; e.g. `plotattr("seriestype")`.


In [28]:
i = 0.0;
while i < 1 
   print(basis(2, i, 3, knotVec))
    i += 0.1
end

[0.0, 1.0, 0.0][0.0, 0.9, 0.1][0.0, 0.8, 0.2][0.0, 0.7, 0.3][0.0, 0.6, 0.4][0.0, 0.5, 0.5][0.0, 0.4, 0.6][0.0, 0.3, 0.7][0.0, 0.2, 0.8][0.0, 0.1, 0.9][0.0, 1.11022e-16, 1.0]